Importing library

In [1]:
import time
import tensorflow as tf
import numpy as np

/opt/anaconda3/envs/GAT_Newest/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/opt/anaconda3/envs/GAT_Newest/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/opt/anaconda3/envs/GAT_Newest/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please update

Set random seed

In [2]:
seed = 123
np.random.seed(seed)
tf.random.set_seed(seed)

Libraries

In [3]:
import sys, os
import csv
from sklearn.metrics import confusion_matrix
import scipy.io as sio
import random
import ABCD_Parser as Reader
import keras

from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import scipy.sparse as sp
import scipy.spatial.distance
import pickle as pkl
import copy
from tqdm import tqdm
from tensorflow.python.ops import array_ops


Libraries for evaluating result

In [4]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import balanced_accuracy_score

Define some functions

In [5]:
def glorot(shape, name = None):
  init_range = np.sqrt(6.0/(shape[0]+shape[1]))
  initial = tf.random_uniform(shape, minval = -init_range, maxval = init_range, dtype = tf.float32)
  var = tf.Variable(initial, name = name)
  return var

In [6]:
def zeros(shape, name = None):
  """All zeros"""
  initial = tf.zeros(shape, dtype=tf.float32)
  return tf.Variable(initial, name = name)

In [7]:
def dot(x,y, sparese = False):
  if sparese:
    res = tf.sparese_tensor_dense_matmul(x,y)
  else:
    res = tf.matmul(x,y)
  return res

In [8]:
def accuracy(preds, labels):
  correct_prediction = tf.equal(tf.round(preds), labels)

  accuracy = tf.cast(correct_prediction, tf.float32)
  return tf.reduce_mean(accuracy)

In [8]:
def tens(shape, name = None):
  initial = tf.constant(10, tf.float32, shape)
  return tf.Variable(initial, name = name)

define flags

In [9]:
from absl import flags, app

FLAGS = flags.FLAGS

flags.DEFINE_integer('node_num', 68, 'Number of Graph nodes')

flags.DEFINE_integer('output_dim', 1, 'Number of output_dim')
flags.DEFINE_float('learning_rate', 0.0001, 'Initial learning rate') #0.0005，0.0001，0.00005，0.00001，0.00003
flags.DEFINE_integer('batch_num', 10, 'Number of epochs to train') #num of batches
flags.DEFINE_integer('epochs', 1000, 'Number of epochs to train') #num of epochs/ iterations through the whole
flags.DEFINE_integer('attn_heads', 5, 'Number of attention head')

flags.DEFINE_integer('hidden1_gat', 24, 'Number of units in hidden layer 1 of gcn') #F_
flags.DEFINE_integer('output_gat', 3, 'Number of units in output layer 1 of gcn') #for later

flags.DEFINE_float('dropout', 0, 'Dropout rate (1 - keep probability).')
flags.DEFINE_float('in_drop', 0, 'Dropout rate (1 - keep probability).')
flags.DEFINE_float('weight_decay', 5e-4, 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_integer('early_stopping', 15, 'Tolerance for early stopping (# of epochs).')


gat_layer

In [10]:
class gat_layer():
  def __init__(self, input_dim,F_, placeholders,attn_heads=1,attn_heads_reduction='concat',
                 activation=tf.nn.relu, use_bias=True,name_=''):
        self.dropout_rate = placeholders['dropout']
        #place holder are like container to be filled later by data
        # dropout for attention matrix aij to regularize attention mechanism
		    # also dropout for feature after transformed to regularize feature transformation
        self.in_drop = placeholders['in_drop']
		    #dropout for input feature X to regularize input representation
        self.name = 'gat_layer'+name_
        self.vars = {}
        # dictionary to store learnable parameter like weights
        self.act = activation
        self.attn_heads = attn_heads  # Number of attention heads (K in the paper)
        self.attn_heads_reduction = attn_heads_reduction  #concat / avg
        self.bias = use_bias
        self.A = placeholders["adj"]
        #adj matrix
        self.input_dim = input_dim

        with tf.variable_scope(self.name+'_vars'):
            for i in range(self.attn_heads): #loops thru all attention head (K)
                self.vars['weights_'+str(i)] = glorot([input_dim, F_], name='weights_' + str(i))
                # init the i-th weights using glorot function
				# the shape of the weight matrix is input_dim x F_ with F_
                self.vars["attn_self_weights_"+str(i)] = glorot([F_, 1], name='attn_self_weights_' + str(i))
				# weight for attention mechanism on itself, shape F_ x 1
                self.vars["attn_neighs_weights_"+str(i)] = glorot([F_, 1], name='attn_neighs_weights_' + str(i))
                # weight for attention mechanism on its neighbor, shape F_ x 1
        if self.bias:
            self.vars['bias'] = zeros([F_],name='bias')

  def __call__(self,inputs):
      #foward pass
      X = inputs
      if self.in_drop != 0.0:
          X = tf.nn.dropout(X, 1-self.in_drop)

      outputs = []
      dense_mask = []

      for head in range(self.attn_heads):
          kernel = self.vars['weights_'+str(head)]
          features = tf.tensordot(X, kernel, axes = 1)
          # project hi to get hi'

          #compute feature combination
          attention_self_kernel = self.vars['attn_self_weights_'+str(head)]
          #weight for attention mechanism (itself)
          attention_neighs_kernel = self.vars['attn_neighs_kernel_'+str(head)]
          #weight for attention mechanism (neighbors)
          attn_for_self = tf.tensordot(features, attention_self_kernel, axes = 1)
          #this give a scalar
          attn_for_neighs = tf.tensordot(features, attention_neighs_kernel, axes = 1)
          #this also give a scalar

          dense = attn_for_self + tf.transpose(attn_for_neighs, [0,2,1]) # N x N

          print("plus:", dense.shape)
          #score e_ij

          #Add nonlinearty
          dense = tf.nn.leaky_relu(dense, alpha = 0.2)

          #Mask with adj matrix to ensure only neighbor
          zero_vec = -9.e15 * tf.ones_like(dense)
          dense = tf.where(self.A > 0, dense, zero_vec)
          dense_mask.append(dense)

          #Apply softmax to get attention coefficient
          dense = tf.nn.softmax(dense) #still NxN
          #Apply dropout to randomly ignore some neighbor regularize
          dropout_attn = tf.nn.dropout(dense, 1-self.dropout_rate) #1-self.drop = drop probability
          #this is the coefficient after dropout
          dropout_feat = tf.nn.dropout(features, 1-self.dropout_rate) #prevent overfitting
          #this is the coefficient after dropout

          #Linear combination with neighbor's features
          node_features = tf.matmul(dropout_attn,dropout_feat)

          if self.bias:
             node_features += self.vars['bias']

          #Add output of attention head to final output
          if self.attn_heads_reduction == 'concat':
              outputs.append(self.act(node_features))
          else:
              outputs.append(node_features)

        #Aggregate the heads's output\
      if self.attn_heads_reduction == 'concat': #concat
        output = tf.concat(outputs, axis = -1)
      else:
          output = tf.add_n(outputs) / self.attn_heads  #average
          output = self.act(output)

      return output, dense_mask

In [12]:
class fc_layer():
  def __init__(self, input_dim, output_dim, placeholders, dropout = 0., sparse_input = False, act = tf.nn.relu, bias = False,
               featureless = False, name_ = ''):
    if dropout:
      self.dropout = placeholders['dropout']
    else:
      self.dropout = 0.

    self.name = 'fc_layer'+ name_
    self.vars = {}
    self.act = act

    self.sparse_input = sparse_input
    self.featureless = featureless
    self.bias = bias

    with tf.variable_scope(self.name + '_vars'):
      self.vars['weights'] = glorot([input_dim, output_dim], name = 'weights')

    if self.bias:
      self.vars['bias'] = zeros([output_dim], name = 'bias')